In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy import genfromtxt
import pywt
from datetime import time, datetime, timedelta
import random
import itertools
import io
import json
import cvxopt

from classification import *
from kernel import *

In [3]:
Xtr = genfromtxt('../data/Xtr.csv', delimiter=',')
Ytr = genfromtxt('../data/Ytr.csv', delimiter=',')
Xte = genfromtxt('../data/Xte.csv', delimiter=',')

Xtr = np.delete(Xtr, 3072, axis=1)
Xte = np.delete(Xte, 3072, axis=1)
Ytr = Ytr[1:,1]
N = len(Ytr)

## Test new SVM

In [4]:
#features_functions = [fourier_modulus_1D_kernel, fourier_modulus_2D_kernel, fourier_phase_2D_kernel, wavelet_transform]
#lambdas = [10, 1, 0.1, 0.01]
features_functions = [fourier_modulus_2D_kernel]
lambdas = [0.1]
submission = False

results_cross_val = {}

for feature_function in features_functions:
    Xtr_t = feature_function(Xtr)
    for _lambda in lambdas:
        results_cross_val[(feature_function, _lambda)] = []
        for i in range(1):
            
            if submission:
                Xte_t = feature_function(Xte)
                features = Xtr_t.T
                labels = Ytr
                X = Xte_t.T
            else:
                mask_test = random.sample(range(0, 5000), 1000)
                mask_train =  [i for i in range(5000) if i not in mask_test]

                Xtr_train = Xtr_t[mask_train, :].T
                Xtr_test = Xtr_t[mask_test, :].T
                Ytr_train = Ytr[mask_train]
                Ytr_test = Ytr[mask_test]
                
                features = Xtr_train
                labels = Ytr_train
                X = Xtr_test

            t1 = datetime.now()
            alphas, bias = one_versus_all_SVM(features, labels, _lambda=_lambda)
            print 'model fitted'
            prediction = predict_SVM(alphas, bias, features, X)
            t2 = datetime.now()
            print t2-t1
            
            if submission == False:
                well_classified = 0
                for i in range(len(prediction)):
                    if prediction[i] == Ytr_test[i]:
                         well_classified+=1
                print float(well_classified)/len(Ytr_test)
            else:
                results_cross_val[(feature_function, _lambda)].append(well_classified)

     pcost       dcost       gap    pres   dres
 0: -4.1980e+02 -1.1594e+01  3e+04  2e+02  9e-11
 1: -8.1069e+00 -1.1484e+01  5e+02  3e+00  6e-11
 2: -1.5490e+00 -1.0267e+01  4e+01  2e-01  3e-12
 3: -9.6340e-01 -5.7431e+00  8e+00  2e-02  5e-13
 4: -8.6031e-01 -2.0262e+00  1e+00  3e-03  1e-13
 5: -9.0130e-01 -1.2899e+00  4e-01  8e-04  1e-13
 6: -9.3299e-01 -1.0788e+00  2e-01  2e-04  1e-13
 7: -9.5252e-01 -9.9984e-01  5e-02  7e-06  1e-13
 8: -9.6182e-01 -9.7626e-01  1e-02  7e-08  1e-13
 9: -9.6499e-01 -9.6961e-01  5e-03  6e-09  1e-13
10: -9.6636e-01 -9.6718e-01  8e-04  6e-10  1e-13
11: -9.6665e-01 -9.6671e-01  7e-05  4e-11  1e-13
12: -9.6667e-01 -9.6667e-01  2e-06  1e-12  1e-13
13: -9.6667e-01 -9.6667e-01  8e-08  5e-14  1e-13
Optimal solution found.
classifier for label  0  done
     pcost       dcost       gap    pres   dres
 0: -3.4004e+02 -1.0850e+01  3e+04  2e+02  6e-11
 1: -6.5647e+00 -1.0760e+01  4e+02  3e+00  5e-11
 2: -1.3297e+00 -9.5842e+00  3e+01  1e-01  3e-12
 3: -8.6434e-01 -

In [ ]:
DF = pd.DataFrame(data=pd.Series(prediction), columns=['Prediction'])
DF.index += 1
DF.to_csv('../data/'+str(feature_function)+'-'+str(_lambda)+'-prediction3.csv', index=True, index_label='Id', sep=',')

## Toy data for tests

In [ ]:
n_samples = 150
mean_1 = [0, -3]
cov = [[3, 0], [0, 3]]
X_1 = np.random.multivariate_normal(mean_1, cov, n_samples).T
mean_2 = [3, 3]
X_2 = np.random.multivariate_normal(mean_2, cov, n_samples).T
mean_3 = [-3, 3]
X_3 = np.random.multivariate_normal(mean_3, cov, n_samples).T
X = np.concatenate((X_1, X_2, X_3), axis = 1)

y = np.concatenate((np.zeros((1,n_samples)), np.ones((1, n_samples)), 2*np.ones((1,n_samples))), axis=1)
y = y[0,:]
Xtr_t = X.T
Ytr = y

mask_test = range(0,450, 5)
mask_train = [i for i in range(450) if i not in mask_test]

Xtr_train = Xtr_t[mask_train, :].T
Xtr_test = Xtr_t[mask_test, :].T
Ytr_train = Ytr[mask_train]
Ytr_test = Ytr[mask_test]